In [ ]:
import numpy as np
import pandas as pd
from NeuralNetwork import loss, activation, layer, network
import plotly.graph_objects as go
import plotly.express as px

# XOR Gate Prediction

In [ ]:
X = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]])

Y = np.array([[0], [1],
              [1], [0]])

In [ ]:
model = network.Sequential()
model.add_layer(layer.Dense(n_neurons=3, name='Layer_1', freeze_weights=False, weights=np.array([[0.1, 0.2, 0.3],
                                                                                                     [0.6, 0.4, 0.7]]),
                                bias=np.array([[0, 0, 0]])))

model.add_layer(activation.Sigmoid(name='Activation_1'))

model.add_layer(layer.Dense(n_neurons=1, name='Layer_2', freeze_weights=False, weights=np.array([[0.1],
                                                                                                     [0.4],
                                                                                                     [0.9]]),
                                bias = np.array([[0]])))

model.add_layer(activation.Sigmoid(name='Activation_2'))

print(model)
model.compile(loss=loss.MSE(), inputs=X, target=Y, batch=4)
print(model)

model.train(epochs=5000)

pred_val = model.predict(inputs=X)
print(pred_val)

# Quantile Regression Prediction Interval
Prediction Interval at 0.977 - 0.023 ~ 0.95

In [ ]:
def f_predictable(x):
    return x+np.sin(np.pi*x/2)


def f(x, std=0.2):
    return f_predictable(x)+np.random.randn(len(x))*std


def get_data(num, start=0, end=4):
        x = np.sort(np.random.rand(num)*(end-start)+start)
        y = f(x)
        return x.reshape(-1, 1), y

x_train, y_train = get_data(num=20000)
y_train = y_train.reshape(-1,1)
x_test, y_test = get_data(num=1000)
y_test = y_test.reshape(-1,1)

In [ ]:
model_upper = network.Sequential()

model_upper.add_layer(layer.Dense(n_neurons=100, name='Layer_1', freeze_weights=False, lr=0.01))

model_upper.add_layer(activation.ReLU(name='Activation_1'))

model_upper.add_layer(layer.Dense(n_neurons=100, name='Layer_2', freeze_weights=False, lr=0.01))

model_upper.add_layer(activation.ReLU(name='Activation_2'))

model_upper.add_layer(layer.Dense(n_neurons=100, name='Layer_3', freeze_weights=False, lr=0.01))

model_upper.add_layer(activation.ReLU(name='Activation_3'))

model_upper.add_layer(layer.Dense(n_neurons=1, name='Layer_4', freeze_weights=False, lr=0.01))

model_upper.compile(loss=loss.Quantile(quantile=0.977), inputs=x_train, target=y_train, batch=24)

model_upper.train(epochs=400)

In [ ]:
model_lower = network.Sequential()

model_lower.add_layer(layer.Dense(n_neurons=100, name='Layer_1', freeze_weights=False, lr=0.01))

model_lower.add_layer(activation.ReLU(name='Activation_1'))

model_lower.add_layer(layer.Dense(n_neurons=100, name='Layer_2', freeze_weights=False, lr=0.01))

model_lower.add_layer(activation.ReLU(name='Activation_2'))

model_lower.add_layer(layer.Dense(n_neurons=100, name='Layer_3', freeze_weights=False, lr=0.01))

model_lower.add_layer(activation.ReLU(name='Activation_3'))

model_lower.add_layer(layer.Dense(n_neurons=1, name='Layer_4', freeze_weights=False, lr=0.01))

model_lower.compile(loss=loss.Quantile(quantile=0.023), inputs=x_train, target=y_train, batch=24)

model_lower.train(epochs=400)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=x_train.reshape(-1), y=y_train.reshape(-1),
                    mode='markers',
                    name='Original Data'))

fig.add_trace(go.Scatter(x=x_test.reshape(-1), y=model_upper.predict(x_test).reshape(-1),
                    mode='lines',
                    name='Upper Bound'))

fig.add_trace(go.Scatter(x=x_test.reshape(-1), y=model_lower.predict(x_test).reshape(-1),
                    mode='lines',
                    name='Lower Bound'))


fig.show()

# Image Classification
Multi Class Prediction

In [ ]:
np.random.seed(42)

cat_images = np.random.randn(700, 2) + np.array([0, -3])
mouse_images = np.random.randn(700, 2) + np.array([3, 3])
dog_images = np.random.randn(700, 2) + np.array([-3, 3])

feature_set = np.vstack([cat_images, mouse_images, dog_images])
labels = np.array([0]*700 + [1]*700 + [2]*700)

one_hot_labels = np.zeros((2100, 3))

for i in range(2100):
    one_hot_labels[i, labels[i]] = 1
    
dataset = pd.DataFrame(np.hstack([feature_set, labels.reshape(-1, 1)]), columns=['X','Y','Labels'])
dataset.Labels = dataset.Labels.astype('str')

In [ ]:
fig = px.scatter(dataset, x="X", y="Y", color="Labels", hover_data=[dataset.index])
fig.update_layout(width=1000, height=800)
fig.show()

Using Cross Entropy

In [ ]:
model = network.Sequential()

model.add_layer(layer.Dense(n_neurons=4, name='Layer_1', freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_1'))

model.add_layer(layer.Dense(n_neurons=3, name='Layer_2', freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_2'))

model.compile(loss=loss.CrossEntropy(), inputs=feature_set, target=one_hot_labels, batch=16)

model.train(epochs=1000)

In [ ]:
pred_val = np.argmax(model.predict(feature_set), axis=1)
np.where((pred_val == labels)==False)

Using SquaredHinge

In [ ]:
one_hot_labels = -1*np.ones((2100, 3))

for i in range(2100):
    one_hot_labels[i, labels[i]] = 1

In [ ]:
model = network.Sequential()

model.add_layer(layer.Dense(n_neurons=4, name='Layer_1', freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_1'))

model.add_layer(layer.Dense(n_neurons=3, name='Layer_2', freeze_weights=False, lr=0.001))

model.add_layer(activation.Tanh(name='Activation_2'))

model.compile(loss=loss.SquaredHinge(), inputs=feature_set, target=one_hot_labels, batch=16)

model.train(epochs=1000)

In [ ]:
pred_val = np.argmax(model.predict(feature_set), axis=1)
np.where((pred_val == labels)==False)

# Image Classification
Multi Label Prediction

In [ ]:
np.random.seed(42)

cat_images = np.random.randn(700, 2) + np.array([0, -3])
mouse_images = np.random.randn(700, 2) + np.array([3, 3])
dog_images = np.random.randn(700, 2) + np.array([-3, 3])

feature_set = np.vstack([cat_images, mouse_images, dog_images])
labels = np.array([0]*700 + [1]*700 + [1]*700)

one_hot_labels = np.zeros((2100, 3))

for i in range(700):
    one_hot_labels[i, labels[i]] = 1
    
for i in range(700, 2100):
    one_hot_labels[i,1:] = 1
    
dataset = pd.DataFrame(np.hstack([feature_set, labels.reshape(-1, 1)]), columns=['X','Y','Labels'])
dataset.Labels = dataset.Labels.astype('str')

In [ ]:
fig = px.scatter(dataset, x="X", y="Y", color="Labels", hover_data=[dataset.index])
fig.update_layout(width=1000, height=800)
fig.show()

Using Cross Entropy

In [ ]:
model = network.Sequential()

model.add_layer(layer.Dense(n_neurons=4, name='Layer_1', freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_1'))

model.add_layer(layer.Dense(n_neurons=3, name='Layer_2', freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_2'))

model.compile(loss=loss.CrossEntropy(), inputs=feature_set, target=one_hot_labels, batch=16)

model.train(epochs=1000)

In [ ]:
pred_val = np.where(model.predict(feature_set)>0.5, 1, 0)
np.where((pred_val == one_hot_labels)==False)

Using SquaredHinge

In [ ]:
one_hot_labels = -np.ones((2100, 3))

for i in range(700):
    one_hot_labels[i, labels[i]] = 1
    
for i in range(700, 2100):
    one_hot_labels[i,1:] = 1

In [ ]:
model = network.Sequential()

model.add_layer(layer.Dense(n_neurons=4, name='Layer_1', freeze_weights=False, lr=0.001))

model.add_layer(activation.Sigmoid(name='Activation_1'))

model.add_layer(layer.Dense(n_neurons=3, name='Layer_2', freeze_weights=False, lr=0.001))

model.add_layer(activation.Tanh(name='Activation_2'))

model.compile(loss=loss.SquaredHinge(), inputs=feature_set, target=one_hot_labels, batch=16)

model.train(epochs=1000)

In [ ]:
pred_val = np.where(model.predict(feature_set)>0, 1, -1)
np.where((pred_val == one_hot_labels)==False)

In [1]:
import numpy as np
from NeuralNetwork import loss

In [2]:
np.random.seed(42)

target = np.random.randint(low=1,high=20, size=(10000,10000))
print(target)

print()

pred = np.random.randint(low=1,high=20, size=(10000,10000))
print(pred)

[[ 7 15 11 ... 10  6 17]
 [17 19 18 ... 14 14 13]
 [16  8  8 ... 15  2  4]
 ...
 [19  4  4 ...  5 18 10]
 [18 17 11 ... 19 12  2]
 [ 9  1  3 ...  8  4  2]]

[[ 7 16 13 ... 19  2  7]
 [13 14  8 ... 12 16 13]
 [12 11 10 ... 17 19  5]
 ...
 [ 9  7  7 ...  6  3 14]
 [ 6 10 11 ...  7 16  3]
 [15 14  8 ...  4 16  1]]


In [3]:
mse = loss.MSE()
mae = loss.MAE()
hubber = loss.Hubber()
logcosh = loss.LogCosh()
quantile = loss.Quantile()
crossentropy = loss.CrossEntropy()
squaredhinge = loss.SquaredHinge()

In [4]:
%timeit mse.forward(target=target, pred_val=pred)
%timeit mse.backward()
print()
%timeit mae.forward(target=target, pred_val=pred)
%timeit mae.backward()
print()
%timeit hubber.forward(target=target, pred_val=pred)
%timeit hubber.backward()
print()
%timeit logcosh.forward(target=target, pred_val=pred)
%timeit logcosh.backward()
print()
%timeit quantile.forward(target=target, pred_val=pred)
%timeit quantile.backward()
print()
%timeit crossentropy.forward(target=target, pred_val=pred)
%timeit crossentropy.backward()
print()
%timeit squaredhinge.forward(target=target, pred_val=pred)
%timeit squaredhinge.backward()

289 ms ± 53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
409 ms ± 40.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

250 ms ± 30.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
533 ms ± 105 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

1.37 s ± 30.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
861 ms ± 32.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

2.45 s ± 27.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.5 s ± 43.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

1.1 s ± 7.05 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
476 ms ± 45.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)



C:\Users\Thinagar\Documents\Python Scripts\DeepLearning\NeuralNetwork\loss.py:214: RuntimeWarning: divide by zero encountered in log
  diff = -self.target * (np.log(self.pred_val)) - (1 - self.target) * np.log(1 - self.pred_val)
C:\Users\Thinagar\Documents\Python Scripts\DeepLearning\NeuralNetwork\loss.py:214: RuntimeWarning: invalid value encountered in log
  diff = -self.target * (np.log(self.pred_val)) - (1 - self.target) * np.log(1 - self.pred_val)
C:\Users\Thinagar\Documents\Python Scripts\DeepLearning\NeuralNetwork\loss.py:214: RuntimeWarning: invalid value encountered in multiply
  diff = -self.target * (np.log(self.pred_val)) - (1 - self.target) * np.log(1 - self.pred_val)


5.94 s ± 62.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


C:\Users\Thinagar\Documents\Python Scripts\DeepLearning\NeuralNetwork\loss.py:226: RuntimeWarning: divide by zero encountered in true_divide
  + ((1 - self.target) / (1 - self.pred_val))
C:\Users\Thinagar\Documents\Python Scripts\DeepLearning\NeuralNetwork\loss.py:226: RuntimeWarning: invalid value encountered in true_divide
  + ((1 - self.target) / (1 - self.pred_val))


1.14 s ± 33.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

526 ms ± 42.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
774 ms ± 41.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
